In [208]:
from collections import defaultdict
import copy

import pint
from pint import UnitRegistry
units = UnitRegistry()
Q_ = units.Quantity

import sigfig

def verify_amount(amount, total=None, maximum=None):
    
    """"checks validity of quantity and unit input when adding/removing/transferring reagents
    if valid returns standardized amount w/units in proper Pint format"""
    
    #note: if amount is not numerical, error message is automatically raised in attempting to compare mag to 0
    #note: units will be automatically checked against unit registry 
        #eventually restrict input to moles or volume only??
        
    if amount <=0:
        raise ValueError("Cannot transfer a negative amount")
    
    elif total!=None: #specific to remove method only
        if total==0:
            raise ValueError("Cannot transfer from an empty container")
            
        elif amount>total:
            raise ValueError("Quantity to be transferred should be less than total quantity in container")
    
    elif maximum!=None: #if max volume of container is specified 
        if amount>maximum:
            raise ValueError("Quantity to be transferred exceeds maximum container volume")
    
    else:
        return amount
        
def round_amount(amount, sigfigs=3):
    
    """"" rounds amount (which must be in proper Pint format) to a default of 3 significant figures"""
    
    mag=sigfig.round(amount.magnitude, sigfigs)
    units=amount.units
    
    return(Q_(mag, units))

In [253]:
import re
from collections import namedtuple

import autoprotocol
import json
from autoprotocol.protocol import Protocol
from autoprotocol.container import Well, WellGroup


class ContainerType(
    namedtuple(
        "ContainerType",
        [
            "name",
            "is_tube",
            "well_count",
            "col_count",
        ],
    )
):
   

    def __new__(
        cls,
        name,
        is_tube,
        well_count,
        col_count,
    ):
        
        
        return super(ContainerType, cls).__new__(
            cls,
            name,
            is_tube,
            well_count,
            col_count,
            
        )

    @staticmethod
    def well_from_coordinates_static(row, row_count, col, col_count):
        if row >= row_count:
            raise ValueError(
                f"0-indexed row {row} is outside of the bounds of {row_count}"
            )

        if col >= col_count:
            raise ValueError(
                f"0-indexed column {col} is outside of the bounds of {col_count}"
            )

        return row * col_count + col

    def well_from_coordinates(self, row, column):
        """
        Gets the well at 0-indexed position (row, column) within the container.
        The origin is in the top left corner.

        Parameters
        ----------
        row : int
            The 0-indexed row index of the well to be fetched
        column : int
            The 0-indexed column index of the well to be fetched

        Returns
        -------
        Int
            The robotized index of the well at at position (row, column)

        Raises
        ------
        ValueError
            if the specified row is outside the bounds of the container_type
        ValueError
            if the specified column is outside the bounds of the container_type
        """
        return ContainerType.well_from_coordinates_static(row, self.row_count(), column, self.col_count)


    @staticmethod
    def robotize_static(well_ref, well_count, col_count):
        if isinstance(well_ref, list):
            return [
                ContainerType.robotize_static(well, well_count, col_count)
                for well in well_ref
            ]

        if not isinstance(well_ref, (str, int, Well)):
            raise TypeError(
                f"ContainerType.robotize(): Well reference "
                f"({well_ref}) given is not of type 'str', 'int', "
                f"or 'Well'."
            )

        if isinstance(well_ref, Well):
            well_ref = well_ref.index
        well_ref = str(well_ref)
        m = re.match(r"([a-z])([a-z]?)(\d+)$", well_ref, re.I)
        if m:
            row = ord(m.group(1).upper()) - ord("A")
            if m.group(2):
                row = 26 * (row + 1) + ord(m.group(2).upper()) - ord("A")
            col = int(m.group(3)) - 1
            row_count = well_count // col_count
            return ContainerType.well_from_coordinates_static(
                row, row_count, col, col_count
            )
        else:
            m = re.match(r"\d+$", well_ref)
            if m:
                well_num = int(m.group(0))
                # Check bounds
                if well_num >= well_count or well_num < 0:
                    raise ValueError(
                        "ContainerType.robotize(): Well number "
                        "given exceeds container dimensions."
                    )
                return well_num
            else:
                raise ValueError(
                    "ContainerType.robotize(): Well must be in "
                    "'A1' format or be an integer.")

    def robotize(self, well_ref):
        """
        Return a robot-friendly well reference from a number of well reference
        formats.

        Example Usage:

        .. code-block:: python

            >>> p = Protocol()
            >>> my_plate = p.ref("my_plate", cont_type="6-flat", discard=True)
            >>> my_plate.robotize("A1")
            0
            >>> my_plate.robotize("5")
            5
            >>> my_plate.robotize(my_plate.well(3))
            3
            >>> my_plate.robotize(["A1", "A2"])
            [0, 1]

        Parameters
        ----------
        well_ref : str, int, Well, list[str or int or Well]
            Well reference to be robotized in string, integer or Well object
            form. Also accepts lists of str, int or Well.

        Returns
        -------
        int or list
            Single or list of Well references passed as row-wise integer
            (left-to-right, top-to-bottom, starting at 0 = A1).

        Raises
        ------
        TypeError
            If well reference given is not an accepted type.
        ValueError
            If well reference given exceeds container dimensions.
        ValueError
            If well reference given is in an invalid format.
        """
        return ContainerType.robotize_static(well_ref, self.well_count, self.col_count)


    @staticmethod
    def humanize_static(well_ref, well_count, col_count):
        ALPHABET = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
        if isinstance(well_ref, list):
            return [
                ContainerType.humanize_static(well, well_count, col_count)
                for well in well_ref
            ]

        if not isinstance(well_ref, (int, str)):
            raise TypeError(
                "ContainerType.humanize(): Well reference given "
                "is not of type 'int' or 'str'."
            )
        try:
            well_ref = int(well_ref)
        except:
            raise TypeError(
                "ContainerType.humanize(): Well reference given"
                "is not parseable into 'int' format."
            )
        # Check bounds
        if well_ref >= well_count or well_ref < 0:
            raise ValueError(
                "ContainerType.humanize(): Well reference "
                "given exceeds container dimensions."
            )
        idx = ContainerType.robotize_static(well_ref, well_count, col_count)
        row, col = (idx // col_count, idx % col_count)
        if row >= len(ALPHABET):
            return ALPHABET[row // 26 - 1] + ALPHABET[row % 26] + str(col + 1)
        else:
            return ALPHABET[row] + str(col + 1)


    def humanize(self, well_ref):
        """
        Return the human readable form of a well index based on the well
        format of this ContainerType.

        Example Usage:

        .. code-block:: python

            >>> p = Protocol()
            >>> my_plate = p.ref("my_plate", cont_type="6-flat", discard=True)
            >>> my_plate.humanize(0)
            'A1'
            >>> my_plate.humanize(5)
            'B3'
            >>> my_plate.humanize('0')
            'A1'

        Parameters
        ----------
        well_ref : int, str, list[int or str]
            Well reference to be humanized in integer or string form.
            If string is provided, it has to be parseable into an int.
            Also accepts lists of int or str

        Returns
        -------
        well_ref : str
            Well index passed as human-readable form.

        Raises
        ------
        TypeError
            If well reference given is not an accepted type.
        ValueError
            If well reference given exceeds container dimensions.

        """
        return ContainerType.humanize_static(well_ref, self.well_count, self.col_count)

    def decompose(self, idx):
        """
        Return the (col, row) corresponding to the given well index.

        Parameters
        ----------
        idx : str or int
            Well index in either human-readable or integer form.

        Returns
        -------
        tuple
            tuple containing the column number and row number of the given
            well_ref.

        Raises
        ------
        TypeError
            Index given is not of the right parameter type

        """
        if not isinstance(idx, (int, str, Well)):
            raise TypeError("Well index given is not of type 'int' or " "'str'.")
        idx = self.robotize(idx)
        return (idx // self.col_count, idx % self.col_count)

    def row_count(self):
        """
        Return the number of rows of this ContainerType.

        """
        return self.well_count // self.col_count    

In [254]:
#common container types

well4 = ContainerType(
    name='4-well PCR plate', is_tube=False, well_count=4,  col_count=2)

tube = ContainerType(
    name='glass tube', is_tube=True, well_count=1,  col_count=0) #any beaker, flask etc

well96 = ContainerType(
    name='96-well PCR plate', is_tube=False, well_count=96,  col_count=8) #8 by 12

In [255]:
class Container(object):
    """
    A reference to a specific physical container (e.g. a tube or 96-well
    microplate).
    Every Container has an associated ContainerType, which defines the well
    count and arrangement, amongst other properties.
    There are several methods on Container which present a convenient interface
    for defining subsets of wells on which to operate. These methods return a
    WellGroup.
    Containers are usually declared using the Protocol.ref method.
    Parameters
    ----------
    id : str, optional
        Alphanumerical identifier for a Container.
    container_type : ContainerType
        ContainerType associated with a Container.
    name : str, optional
        name of the container/ref being created.
    storage : str, optional
        name of the storage condition.
    cover : str, optional
        name of the cover on the container.
    Raises
    ------
    AttributeError
        Invalid cover-type given
    """

    def __init__(self, name, container_type):
        self.name = name
        self.container_type = container_type

        self._wells = [Well(self, idx) for idx in range(container_type.well_count)]
        

    def well(self, i):
        """
        Return a Well object representing the well at the index specified of
        this Container.
        Parameters
        ----------
        i : int, str
            Well reference in the form of an integer (ex: 0) or human-readable
            string (ex: "A1").
        Returns
        -------
        Well
            Well for given reference
        Raises
        ------
        TypeError
            index given is not of the right type
        """
        if not isinstance(i, (int, str)):
            raise TypeError("Well reference given is not of type 'int' or " "'str'.")
        return self._wells[self.robotize(i)]

    def well_from_coordinates(self, row, column):
        """
        Gets the well at 0-indexed position (row, column) within the container.
        The origin is in the top left corner.
        Parameters
        ----------
        row : int
            The 0-indexed row index of the well to be fetched
        column : int
            The 0-indexed column index of the well to be fetched
        Returns
        -------
        Well
            The well at position (row, column)
        """
        return self.well(
            self.container_type.well_from_coordinates(row=row, column=column)
        )

    def tube(self):
        """
        Checks if container is tube and returns a Well representing the zeroth
        well.
        Returns
        -------
        Well
            Zeroth well of tube
        Raises
        -------
        AttributeError
            If container is not tube
        """
        if self.container_type.is_tube:
            return self.well(0)
        else:
            raise AttributeError(
                f"{self} is a {self.container_type.shortname} " f"and is not a tube"
            )

    def wells(self, *args):
        """
        Return a WellGroup containing references to wells corresponding to the
        index or indices given.
        Parameters
        ----------
        args : str, int, list
            Reference or list of references to a well index either as an
            integer or a string.
        Returns
        -------
        WellGroup
            Wells from specified references
        Raises
        ------
        TypeError
            Well reference is not of a valid input type
        """
        if isinstance(args[0], list):
            wells = args[0]
        else:
            wells = [args[0]]
        for a in args[1:]:
            if isinstance(a, list):
                wells.extend(a)
            else:
                wells.extend([a])
        for w in wells:
            if not isinstance(w, (str, int, list)):
                raise TypeError(
                    "Well reference given is not of type" " 'int', 'str' or 'list'."
                )

        return WellGroup([self.well(w) for w in wells])

    def robotize(self, well_ref):
        """
        Return the integer representation of the well index given, based on
        the ContainerType of the Container.
        Uses the robotize function from the ContainerType class. Refer to
        `ContainerType.robotize()` for more information.
        """
        if not isinstance(well_ref, (str, int, Well, list)):
            raise TypeError(
                "Well reference given is not of type 'str' " "'int', 'Well' or 'list'."
            )
        return self.container_type.robotize(well_ref)

    def humanize(self, well_ref):
        """
        Return the human readable representation of the integer well index
        given based on the ContainerType of the Container.
        Uses the humanize function from the ContainerType class. Refer to
        `ContainerType.humanize()` for more information.
        """
        if not isinstance(well_ref, (int, str, list)):
            raise TypeError(
                "Well reference given is not of type 'int'," "'str' or 'list'."
            )
        return self.container_type.humanize(well_ref)

    def decompose(self, well_ref):
        """
        Return a tuple representing the column and row number of the well
        index given based on the ContainerType of the Container.
        Uses the decompose function from the ContainerType class. Refer to
        `ContainerType.decompose()` for more information.
        """
        if not isinstance(well_ref, (int, str, Well)):
            raise TypeError(
                "Well reference given is not of type 'int', " "'str' or Well."
            )
        return self.container_type.decompose(well_ref)

    def all_wells(self, columnwise=False):
        """
        Return a WellGroup representing all Wells belonging to this Container.
        Parameters
        ----------
        columnwise : bool, optional
            returns the WellGroup columnwise instead of rowwise (ordered by
            well index).
        Returns
        -------
        WellGroup
            WellGroup of all Wells in Container
        """
        if columnwise:
            num_cols = self.container_type.col_count
            num_rows = self.container_type.well_count // num_cols
            return WellGroup(
                [
                    self._wells[row * num_cols + col]
                    for col in range(num_cols)
                    for row in range(num_rows)
                ]
            )
        else:
            return WellGroup(self._wells)

    def inner_wells(self, columnwise=False):
        """
        Return a WellGroup of all wells on a plate excluding wells in the top
        and bottom rows and in the first and last columns.
        Parameters
        ----------
        columnwise : bool, optional
            returns the WellGroup columnwise instead of rowwise (ordered by
            well index).
        Returns
        -------
        WellGroup
            WellGroup of inner wells
        """
        num_cols = self.container_type.col_count
        num_rows = self.container_type.row_count()
        inner_wells = []
        if columnwise:
            for c in range(1, num_cols - 1):
                wells = []
                for r in range(1, num_rows - 1):
                    wells.append((r * num_cols) + c)
                inner_wells.extend(wells)
        else:
            well = num_cols
            for _ in range(1, num_rows - 1):
                inner_wells.extend(range(well + 1, well + (num_cols - 1)))
                well += num_cols
        inner_wells = [self._wells[x] for x in inner_wells]
        return WellGroup(inner_wells)

    def wells_from(self, start, num, columnwise=False):
        """
        Return a WellGroup of Wells belonging to this Container starting from
        the index indicated (in integer or string form) and including the
        number of proceeding wells specified. Wells are counted from the
        starting well rowwise unless columnwise is True.
        Parameters
        ----------
        start : Well or int or str
            Starting well specified as a Well object, a human-readable well
            index or an integer well index.
        num : int
            Number of wells to include in the Wellgroup.
        columnwise : bool, optional
            Specifies whether the wells included should be counted columnwise
            instead of the default rowwise.
        Returns
        -------
        WellGroup
            WellGroup of selected wells
        Raises
        ------
        TypeError
            Incorrect input types, e.g. `num` has to be of type int
        """
        if not isinstance(start, (str, int, Well)):
            raise TypeError(
                "Well reference given is not of type 'str'," "'int', or 'Well'."
            )
        if not isinstance(num, int):
            raise TypeError("Number of wells given is not of type 'int'.")

        start = self.robotize(start)
        if columnwise:
            row, col = self.decompose(start)
            num_rows = self.container_type.row_count()
            start = col * num_rows + row
        return WellGroup(self.all_wells(columnwise).wells[start : start + num])


In [286]:
"""Classes modelling chemicals and substances
"""

"""
material - solid, liquid, or gas reagent from which we can make a mixture.
stored with its properties (molar mass and density would probably be useful) and phase (state of matter)
"""

class MaterialModel(): 
    
    def __init__(self, name, phase, properties=None): 
        
        self.name = name ##we should standardize the input format for names eventually
        
        self.properties = properties #optional... can include density, molar mass, etc
        
        # phase is a state of matter (solid, liquid, gas) ... 
        #to distinguish from the more common "state" of system at any given time
        self.phase = phase 
        
    def __repr__(self): 
        return f'{self.name}'
    
    def __hash__(self): 
        return hash(self.name)
    

class MixtureModel(): 
  
    def __init__(self, phase=None):
        """        
        You can add components to the mixture, but can't remove them
        (unless you remove a proportion of the mixture, assuming it is liquid and ideal)
        """
        
        self.phase= phase 
        #if we want to specify the phase of the overall mixture to define behavior/properties
        #e.g. if aqeuous, capable of liquid transfer
        #otherwise takes None as default
        
        self.materials= defaultdict(lambda: 0)
         
            #there is  a single dictionary for all the phases of matter, and the entire material model is stored
    #so you can index by phase of matter if you desire, via a loop across the keys
    #but it makes doing math with the dicitonaries much more concise
        
    
    def add(self, amounts):
                        
        for mat, amount in amounts.items():
            a= verify_amount(amount)
            self.materials[mat] += a
                
 
    def remove(self, amount):
        
        #assumes ideal mixture and removes proportional amount of everything
        #eventually extraction can be incorporated 
        
        if self.phase!='aqueous': 
            raise TypeError(f"Method not supported for this type of mixture")
            #only works for aqueous mixtures
        
        else: 
            self.removed_mixture = defaultdict(lambda: 0)
        
            initial_conc = self.conc.copy()
            initial_vol = self.total_volume
            
            a=verify_amount(amount, initial_vol)
            
            for mat, val in self.materials.items():
                if mat.phase!='liquid': #solids and gases
                    self.removed_mixture[mat] += round_amount((a * initial_conc[mat].to(units.molar)).to(units.mol))
                    self.materials[mat] -= round_amount((a * initial_conc[mat].to(units.molar)).to(units.mol))
                elif mat.phase=='liquid':
                    self.removed_mixture[mat] += round_amount((val * a/initial_vol).to_reduced_units())
                    self.materials[mat] -= round_amount((val * a/initial_vol).to_reduced_units())
            return self.removed_mixture
        
    
    @property
    def total_measure(self):
        return sum(list(self.materials.values()))
        # adds up total amount (moles) of substance in the mixture 
   
        
    @property
    def total_volume(self):
        
        if self.phase!='aqueous': 
            raise TypeError(f"Invalid property for this type of mixture")
            #only meaningful for aqueous mixtures
        
        else: 
            vol=0
            for mat, amount in self.materials.items():
                if mat.phase=='liquid':
                    vol+= amount
            return round_amount(vol)

    @property
    def conc(self): 
        
        self.concentrations=defaultdict(lambda: 0)
        
        if self.phase=='aqueous': #aqueous solutions, return molarity
            for key, val in self.materials.items(): 
                if key.phase!='liquid':
                    self.concentrations[key] = round_amount(val/self.total_volume.to(units.liter))
    
            return self.concentrations
    
        else: #all other solutions, return mole fraction
    
            for key, val in self.materials.items(): 
                self.concentrations[key] = round_amount(val/self.total_measure)

            return self.concentrations
        
    def __repr__(self): 
        
        mat_names = [m.name for m in self.materials.keys()] 
        return f"Mixture of {', '.join(mat_names)}"
        

    def __len__(self):  #total number of components added
        return len(self.materials)

    def __hash__(self): 
        return hash(str(self.materials) + str(id(self)))
    
class ContainerModel():
    
    def __init__(self, name, idx=0, max_vol=None, contents = None, temp = None): 
        
        self.name=name
        self.idx=idx
        self.max_vol=max_vol
        self.contents=contents
        self.temp= temp

    def __repr__(self): 
        return f'{self.name}'
    
    def __hash__(self): 
        return hash(self.name)
    
class WorkspaceTemplate():
    
    def __init__(self): 
        self.steps=[]
    
    def record(self, containers, actions=None):
        
        self.steps.append([containers, actions])
    
        return self.steps
    
class HeatAction():
    
    def __init__(self, container, f_temp): 
        self.container = container
        self.f_temp = f_temp
    
    def do(self):
        self.container.temp = self.f_temp
        return f'Heated {self.container} to {self.f_temp}' 

class VortexAction():
    
    def __init__(self, container, rpm, time): 
        self.container = container
        self.rpm = rpm
        self.time = time
        
    def do(self): 
        return f'Vortexed {self.container} at {self.rpm} rpm for {self.time}' 

        
class TransferAction():

    def __init__(self, container_from, container_to, amount): 
        self.container_from = container_from
        self.container_to = container_to
        self.amount = amount
    

    def do(self):
        self.container_to.contents.add(self.container_from.contents.remove(self.amount))
        return f'Transfered {self.amount} from {self.container_from} to {self.container_to}'
        

In [289]:
HCOOH = MaterialModel(name='formic acid', phase= 'liquid')
GBL = MaterialModel(name='GBL', phase= 'liquid')
PbI2 = MaterialModel(name='lead iodide', phase= 'solid')

stock = MixtureModel('aqueous')
stock.add({GBL: Q_(20, 'mL'), HCOOH: Q_(30, 'mL'), PbI2: Q_(0.2, 'mol')})

In [290]:
exp1wells=Container('exp1wells', well4)

In [291]:
beaker=ContainerModel(name='stock beaker', max_vol=Q_(250, 'mL'), contents= stock)

In [293]:
perovskite_workspace= WorkspaceTemplate() #workspace will contain the stock beaker and the 4 wells

containers = [beaker]

for well in exp1wells.all_wells():
    index=well.humanize()
    i=ContainerModel(name='well{}'.format(index), idx=index, max_vol=Q_(1, 'mL'), contents = None, temp = Q_(30, 'degC'))
    containers.append(i)

containers

[stock beaker, wellA1, wellA2, wellB1, wellB2]

In [294]:
perovskite_workspace.record(containers) 

#records all containers at step 0, no actions performed yet

[[[stock beaker, wellA1, wellA2, wellB1, wellB2], None]]

In [302]:
type(beaker)

__main__.ContainerModel

In [303]:
action1=(TransferAction(beaker, containers[0], Q_(1, 'mL')).do()) #transfer 1 mL of stock to well 1
    
perovskite_workspace.record(containers, action1) 

AttributeError: 'NotImplementedType' object has no attribute '_REGISTRY'

In [305]:
#vortex all the wells 

action2=(VortexAction(containers[1], 750, Q_(15, 'minute')).do())
perovskite_workspace.record(containers, action2)

[[[stock beaker, wellA1, wellA2, wellB1, wellB2], None],
 [[stock beaker, wellA1, wellA2, wellB1, wellB2],
  'Vortexed wellA1 at 750 rpm for 15 minute']]

In [204]:
well=(wells.well_from_coordinates(1,0)).humanize()

action2=(VortexAction(well, 750, Q_(15, 'minute')).do())
perovskite_workspace.record(containers, action2)

[[[stock beaker of type glass tube, well plate of type 96-well PCR plate],
  None],
 [[stock beaker of type glass tube, well plate of type 96-well PCR plate],
  'Vortexed well plate of type 96-well PCR plate at 750 rpm for 15 minute'],
 [[stock beaker of type glass tube, well plate of type 96-well PCR plate],
  'Vortexed Well(well plate of type 96-well PCR plate, 8, None) at 750 rpm for 15 minute'],
 [[stock beaker of type glass tube, well plate of type 96-well PCR plate],
  'Vortexed B1 at 750 rpm for 15 minute'],
 [[stock beaker of type glass tube, well plate of type 96-well PCR plate],
  'Vortexed B1 at 750 rpm for 15 minute']]

In [207]:
well_group=[]
for row in range(8):
    well_group.append((wells.well_from_coordinates(row,0)).humanize()) #all wells in column 1

#heat column 1

action3=[]
for well in well_group:
    action3.append(HeatAction(well, Q_(70, 'degC')).do())

perovskite_workspace.record(containers, action3)

AttributeError: 'str' object has no attribute 'temp'

In [ ]:
HCOOH = MaterialModel(name='formic acid', phase= 'liquid', properties={'molar mass': Q_(46.03, 'g/mol'), 'density': Q_(1.22, 'g/mL')})
GBL = MaterialModel(name='GBL', phase= 'liquid', properties={'molar mass': Q_(86.09, 'g/mol'), 'density': Q_(1.13, 'g/mL')})
PbI2 = MaterialModel(name='lead iodide', phase= 'solid', properties={'molar mass': Q_(461.01, 'g/mol')})

In [ ]:
def convert(substance, quantity, units):
    
    for key, val in quantity.dimensionality.items():
        unit_type= key
        dimension = val
    
    if units == 'mole': 
        if unit_type == '[mass]': #mass to mole conversions
            return quantity / substance.properties['molar mass']
        
        elif unit_type == '[length]' and dimension==3 : #volume to mole conversions
            return quantity * substance.properties['density'] / substance.properties['molar mass']
    
    elif units == 'mass':
        return quantity * substance.properties['molar mass'] #moles to mass
    
    elif units == 'volume':
        return quantity * substance.properties['molar mass'] / substance.properties['density']
        #moles to volume


In [ ]:
convert(PbI2, Q_(10, 'mol'), 'mass')

In [ ]:
amounts = {GBL: Q_(10, 'mL'), HCOOH: Q_(3, 'mL'), PbI2: Q_(231, 'g')}

stock2.add(amounts)

In [ ]:
for key,val in amounts.items():
    to_moles=convert(key, val, 'mole')
    amounts[key]=to_moles

In [ ]:
stock2.add(amounts)

In [ ]:
stock2.materials, stock2.conc

In [ ]:
volumes=[]
moles={}
for key,val in amounts.items():
    if key.phase=='liquid':
        to_vol=convert(key, val, 'volume')
        volumes.append(to_vol)
    if key.phase=='solid':
        moles[key] = val


In [ ]:
sum(volumes)

In [ ]:
molarities={}
for key, val in moles.items():
    molarities[key] = (val/sum(volumes)).to(units.molar)

molarities

In [ ]:
stock2.add({GBL: 10})

In [ ]:
stock2.add({GBL: Q_(-10, 'mol')})

In [ ]:
stock.materials, stock.total_measure

In [ ]:
Q_(.003, 'mol')

In [ ]:
stock.remove(Q_(.003, 'mol'))

In [ ]:
stock = MixtureModel()
stock.add({GBL: Q_(1, 'mol'), HCOOH: Q_(3, 'mol'), PbI2: Q_(0.2, 'mol')})
stock.materials

In [ ]:
stock.remove(Q_(.3, 'mol'))

In [ ]:
stock.materials

In [ ]:
a = (Q_(5, 'mol') / Q_(2, 'mol'))
Q_(a.magnitude, 'mol')

In [ ]:
Q_(5, 'mL').to(units('L'))

In [ ]:
stock.materials

In [ ]:
stock.total_measure

In [ ]:
stock.remove(Q_(4, 'mol'))

In [ ]:
HCOOH = MaterialModel(name='formic acid', phase= 'liquid', properties={'molar mass': Q_(46.03, 'g/mol'), 'density': Q_(1.22, 'g/mL')})
GBL = MaterialModel(name='GBL', phase= 'liquid', properties={'molar mass': Q_(86.09, 'g/mol'), 'density': Q_(1.13, 'g/mL')})
PbI2 = MaterialModel(name='lead iodide', phase= 'solid', properties={'molar mass': Q_(461.01, 'g/mol')})

stock=MixtureModel()
amounts= {GBL: Q_(10, 'mL'), HCOOH: Q_(3, 'mL'), PbI2: Q_(231, 'g')}

    
for key,val in amounts.items():
    to_moles=convert(key, val, 'mole')
    amounts[key]=to_moles
        
amounts

stock.add(amounts)

stock.materials, stock.total_measure, stock.conc

In [ ]:
volumes={}
moles={}
masses={}
for key,val in stock.materials.items():
    if key.phase=='liquid':
        to_vol=convert(key, val, 'volume')
        volumes[key]=to_vol
    if key.phase=='solid':
        moles[key] = val
        to_mass=convert(key, val, 'mass')
        masses[key] = to_mass



volumes, moles, masses



In [ ]:
molarities={}
for key, val in moles.items():
    molarities[key] = (val/sum(val for key, val in volumes.items())).to(units.molar)
{**moles, **volumes}.items()

In [ ]:
removed={}

amount=Q_(1, 'mL')
for mat, val in {**moles, **volumes}.items():
    if mat.phase=='solid':
        removed[mat] += (amount * molarities[mat].to(units.molar)).to(units.mol)
        self.materials[mat] -= (amount * initial_conc[mat].to(units.molar)).to(units.mol)
    elif mat.phase=='liquid':
        self.removed_mixture[mat] += (val * amount/initial_vol).to_reduced_units() 
        self.materials[mat] -= (val * amount/initial_vol).to_reduced_units() 

        
removed